In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [23]:
base = pd.read_csv('test.csv')

In [24]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [25]:
x.head()

,Amostra 1,Amostra 2,Amostra 3,Amostra 4,Amostra 5,Amostra 6,Amostra 7,Amostra 8,Amostra 9,Amostra 10,...,Amostra 991,Amostra 992,Amostra 993,Amostra 994,Amostra 995,Amostra 996,Amostra 997,Amostra 998,Amostra 999,Amostra 1000
0,0.989,0.985,0.981,0.976,0.970,0.963,0.956,0.949,0.940,0.931,...,0.997,0.998,1.000,1.000,1.000,0.999,0.998,0.995,0.993,0.989
1,0.282,0.170,0.057,-0.056,-0.169,-0.282,-0.393,-0.504,-0.612,-0.719,...,1.212,1.120,1.025,0.926,0.824,0.720,0.613,0.504,0.394,0.282
2,3.637,3.572,3.501,3.426,3.346,3.260,3.171,3.076,2.978,2.875,...,3.984,3.968,3.946,3.918,3.885,3.846,3.802,3.752,3.697,3.637
3,0.564,0.340,0.114,-0.113,-0.339,-0.564,-0.787,-1.008,-1.225,-1.438,...,2.424,2.240,2.049,1.851,1.648,1.439,1.226,1.008,0.788,0.564
4,-0.559,-0.474,-0.388,-0.301,-0.214,-0.126,-0.038,0.050,0.138,0.226,...,-1.257,-1.187,-1.115,-1.041,-0.965,-0.887,-0.807,-0.726,-0.643,-0.559


In [26]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(150,)

In [27]:
y = y.values.reshape(-1,1)
y.shape

(150, 1)

In [28]:
y = onehot.fit_transform(y).toarray()
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [29]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [30]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((150, 1000), (45, 1000), (150, 3), (45, 3))

In [31]:
neuronios_entrada = x.shape[1]
neuronios_entrada

1000

In [32]:
neuronios_oculta = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta

501

In [33]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [34]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable:0' shape=(1000, 501) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_1:0' shape=(501, 3) dtype=float32_ref>}

In [35]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_2:0' shape=(501,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_3:0' shape=(3,) dtype=float32_ref>}

In [36]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [37]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [38]:
modelo = mlp(xph, w, b)

In [39]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [78]:
batch_size = 10

In [79]:
batch_total = int(len(x_treinamento) / batch_size)
batch_total

10

In [80]:
x_batches = np.array_split(x_treinamento, batch_total)

In [81]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(5000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])

Época: 1 erro: 2438.7977172851565
Época: 501 erro: 9.854802328845835e-06
Época: 1001 erro: 7.993385679583298e-07
Época: 1501 erro: 6.664873595241262e-08
Época: 2001 erro: 4.551626808790843e-09
Época: 2501 erro: 0.0
Época: 3001 erro: 0.0
Época: 3501 erro: 0.0
Época: 4001 erro: 0.0
Época: 4501 erro: 0.0


In [67]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

In [82]:
y_teste2 = np.argmax(y_teste, 1)

In [83]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.6666666666666666